This notebook

In [1]:
import geopandas as gpd
import yaml
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import asf_search as asf
from datetime import timedelta
from dateparser import parse
import concurrent.futures
import hyp3_sdk
import numpy as np

# Setup

In [2]:
YAML_FILE = 'enumeration_parameters.yml'
with open(YAML_FILE) as f:
    enum_params = yaml.safe_load(f)['enumeration_parameters']

In [3]:
# AOI Name
AOI_NAME = enum_params['aoi_name']
ENUM_DIR = enum_params['enum_directory']

In [4]:
enum_dir = Path(ENUM_DIR)
assert enum_dir.exists()

In [5]:
AOI_NAME, ENUM_DIR

('mainland_us_coast', 'out/enum/mainland_us_coast_thru_202305_n2')

# Read enumeration data

In [6]:
dfs = [gpd.read_file(p) for p in tqdm(list(enum_dir.glob('*.geojson')))]
df_ifg = pd.concat(dfs, axis=0)
df_ifg.head()

100%|███████████████████████████████████████████████████████████████████████████████████████| 65/65 [00:11<00:00,  5.56it/s]


,reference,secondary,reference_date,secondary_date,frame_id,track_key,aoi_name,neighbors,geometry
0,S1B_IW_SLC__1SDV_20211213T010745_20211213T0108...,S1B_IW_SLC__1SDV_20211201T010746_20211201T0108...,2021-12-13,2021-12-01,7489.0,track49,mainland_us_coast,2.0,"POLYGON Z ((-107.85434 30.85203 0.00000, -107...."
1,S1B_IW_SLC__1SDV_20211213T010745_20211213T0108...,S1B_IW_SLC__1SDV_20211201T010746_20211201T0108...,2021-12-13,2021-12-01,7490.0,track49,mainland_us_coast,2.0,"POLYGON Z ((-108.11447 32.01369 0.00000, -108...."
2,S1B_IW_SLC__1SDV_20211213T010810_20211213T0108...,S1B_IW_SLC__1SDV_20211201T010811_20211201T0108...,2021-12-13,2021-12-01,7491.0,track49,mainland_us_coast,2.0,"POLYGON Z ((-108.37802 33.17707 0.00000, -108...."
3,S1B_IW_SLC__1SDV_20211213T010835_20211213T0109...,S1B_IW_SLC__1SDV_20211201T010835_20211201T0109...,2021-12-13,2021-12-01,7492.0,track49,mainland_us_coast,2.0,"POLYGON Z ((-108.64460 34.33971 0.00000, -108...."
4,S1B_IW_SLC__1SDV_20211213T010900_20211213T0109...,S1B_IW_SLC__1SDV_20211201T010900_20211201T0109...,2021-12-13,2021-12-01,7493.0,track49,mainland_us_coast,2.0,"POLYGON Z ((-108.91826 35.51715 0.00000, -109...."


# Deduplication of IFGs CMR

See this [PR]() for the generation of this feature.

The raw CMR command for a comparable query is:
```
https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?short_name=SENTINEL-1_INTERFEROGRAMS&temporal=2022-02-24T00:00:00Z,2022-02-25T00:00:00Z&attribute[]=int,FRAME_NUMBER,25502&attribute[]=int,TEMPORAL_BASELINE_DAYS,12
```

In [10]:
def get_gunw_hits(record):

    frame_id = int(record['frame_id'])
    ref_date = parse(record['reference_date'])
    sec_date = parse(record['secondary_date'])
    
    start = ref_date - timedelta(days=.5)
    end = ref_date + timedelta(days=.5)
    tb_days = (ref_date - sec_date).days
    

    opts = asf.ASFSearchOptions(host='cmr.uat.earthdata.nasa.gov', 
                                platform=asf.SENTINEL1,
                                processingLevel=asf.constants.GUNW_STD,
                                frame=frame_id, 
                                start=start, 
                                end=end, 
                                temporalBaselineDays=[tb_days],
                                maxResults=5
                                )
    
    results = asf.search(opts=opts)

    return len(results)

In [11]:
submission_records = df_ifg.to_dict('records')
#get_gunw_hits(submission_records[0])

In [12]:
n = len(submission_records)
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    ifg_hits_cmr = list(tqdm(executor.map(get_gunw_hits, submission_records[:100]), total=n))

  0%|                                                                                | 100/149448 [00:19<8:02:42,  5.16it/s]


In [13]:
submission_records[0]

{'reference': 'S1B_IW_SLC__1SDV_20211213T010745_20211213T010812_030000_0394DC_75EC S1B_IW_SLC__1SDV_20211213T010810_20211213T010837_030000_0394DC_11D6',
 'secondary': 'S1B_IW_SLC__1SDV_20211201T010746_20211201T010813_029825_038F5F_EC75 S1B_IW_SLC__1SDV_20211201T010811_20211201T010838_029825_038F5F_1F47',
 'reference_date': '2021-12-13',
 'secondary_date': '2021-12-01',
 'frame_id': 7489.0,
 'track_key': 'track49',
 'aoi_name': 'mainland_us_coast',
 'neighbors': 2.0,
 'geometry': <POLYGON Z ((-107.854 30.852 0, -107.895 31.03 0, -107.929 31.197 0, -108.08...>}

In [14]:
ifg_hits_cmr[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
# TODO: set to ifg_hits_cmr
df_ifg['cmr_hits'] = 0

# Format Job Parameters

In [16]:
IONO_CORRECTION = True
SET_CORRECTION = True
weather_model = 'HRRR'

In [17]:
def generate_job_name(record):
    track_key = record['track_key']
    aoi_name = record['aoi_name']
    neighbor = str(int(record['neighbors']))
    return f'{track_key}-n{neighbor}'

In [18]:
job_names = list(map(generate_job_name, submission_records))
job_names_unique = list(set(job_names))
job_names_unique[:5]

['track92-n2', 'track158-n2', 'track86-n2', 'track165-n2', 'track173-n2']

In [19]:
job_parameters = [{'granules': record['reference'].split(' '),
                   'secondary_granules': record['secondary'].split(' '),
                   'frame_id': int(record['frame_id']),
                   'weather_model': weather_model,
                   'estimate_ionosphere_delay': IONO_CORRECTION,
                   'compute_solid_earth_tide': SET_CORRECTION,
                  } for record in submission_records]

job_dicts = [{'name': job_name,
              # NOTE: we are still using the `dev` branch. Change this to "INSAR_ISCE" to use the `main` branch.
              'job_type': 'INSAR_ISCE_TEST',
              'job_parameters': parameters
             }
             for parameters, job_name in zip(job_parameters, job_names) ]
job_dicts[:2]

[{'name': 'track49-n2',
  'job_type': 'INSAR_ISCE_TEST',
  'job_parameters': {'granules': ['S1B_IW_SLC__1SDV_20211213T010745_20211213T010812_030000_0394DC_75EC',
    'S1B_IW_SLC__1SDV_20211213T010810_20211213T010837_030000_0394DC_11D6'],
   'secondary_granules': ['S1B_IW_SLC__1SDV_20211201T010746_20211201T010813_029825_038F5F_EC75',
    'S1B_IW_SLC__1SDV_20211201T010811_20211201T010838_029825_038F5F_1F47'],
   'frame_id': 7489,
   'weather_model': 'HRRR',
   'estimate_ionosphere_delay': True,
   'compute_solid_earth_tide': True}},
 {'name': 'track49-n2',
  'job_type': 'INSAR_ISCE_TEST',
  'job_parameters': {'granules': ['S1B_IW_SLC__1SDV_20211213T010745_20211213T010812_030000_0394DC_75EC',
    'S1B_IW_SLC__1SDV_20211213T010810_20211213T010837_030000_0394DC_11D6'],
   'secondary_granules': ['S1B_IW_SLC__1SDV_20211201T010746_20211201T010813_029825_038F5F_EC75',
    'S1B_IW_SLC__1SDV_20211201T010811_20211201T010838_029825_038F5F_1F47'],
   'frame_id': 7490,
   'weather_model': 'HRRR',
   

# Submit Jobs to Hyp3

In [20]:
# uses .netrc; add `prompt=True` to prompt for credentials; 
hyp3_isce = hyp3_sdk.HyP3('https://hyp3-a19-jpl.asf.alaska.edu')

In [21]:
N_batches = int(np.ceil(len(job_dicts) / 200))
job_dicts_batches = [job_dicts[200 * k : 200 * (k + 1)] for k in range(N_batches)]

In [22]:
## For testing
# submitted_jobs = hyp3_isce.submit_prepared_jobs(job_dicts_batches[0][:])

In [23]:
# for job_dict in tqdm(job_dicts_batches):
#     submitted_jobs = hyp3_isce.submit_prepared_jobs(job_dicts[:])
#     continue

100%|████████████████████████████████████████████████████████████████████████████████| 748/748 [00:00<00:00, 2586429.84it/s]


Check status

In [25]:
for job_name in job_names_unique[:5]:
    jobs = hyp3_isce.find_jobs(name=job_name)
    print('#' * 10)
    print(job_name)
    print(jobs)

##########
track92-n2
0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.
##########
track158-n2
0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.
##########
track86-n2
0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.
##########
track165-n2
0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.
##########
track173-n2
0 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 0 pending.
